In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import Image, display

# URL der Seite mit der Pokémon-Tabelle
url = 'https://pokemondb.net/pokedex/all'

# Senden einer GET-Anfrage an die Webseite
response = requests.get(url)

# BeautifulSoup-Objekt zur Analyse des HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Finden der Tabelle mit Pokémon-Daten
table = soup.find('table', {'class': 'data-table'})

# Extrahieren der Kopfzeile der Tabelle
headers = [th.text.strip() for th in table.find_all('th')]

# Extrahieren der Zeilen der Tabelle
rows = []
sprite_urls = []
for tr in table.find_all('tr')[1:]:  # überspringe die Kopfzeile
    cols = tr.find_all('td')
    row = [col.text.strip() for col in cols]
    rows.append(row)
    
    # Finden des Sprite-Links für jedes Pokémon (das Bild ist im ersten td-Tag enthalten)
    sprite_tag = cols[0].find('img')
    if sprite_tag:
        sprite_url = sprite_tag['src']
        sprite_urls.append(f"https:{sprite_url}")
    else:
        sprite_urls.append(None)

# Erstellen eines Pandas DataFrames aus den extrahierten Daten
pokemon_df = pd.DataFrame(rows, columns=headers)

# Hinzufügen der Sprite-URLs zum DataFrame
pokemon_df['Sprite'] = sprite_urls

# Anzeigen des DataFrames
print(pokemon_df)

# Anzeigen der Sprites direkt im Notebook
for sprite_url in sprite_urls[:5]:  # Anzeigen der ersten 5 Pokémon-Sprites
    if sprite_url:
        display(Image(url=sprite_url))

# Datentypen konvertieren
numeric_cols = ['#', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
pokemon_df[numeric_cols] = pokemon_df[numeric_cols].apply(pd.to_numeric)

# Aufteilen der Typen
pokemon_df['Type 1'] = pokemon_df['Type'].str.split().str[0]
pokemon_df['Type 2'] = pokemon_df['Type'].str.split().str[1]

# 1. Stärkstes Pokémon für jeden Typ
def get_strongest_per_type(df):
    types = pd.concat([df['Type 1'], df['Type 2']]).dropna().unique()
    strongest = []
    for t in types:
        strongest_pokemon = df[(df['Type 1'] == t) | (df['Type 2'] == t)].sort_values('Total', ascending=False).iloc[0]
        strongest.append((t, strongest_pokemon['Name'], strongest_pokemon['Total']))
    return pd.DataFrame(strongest, columns=['Type', 'Strongest Pokémon', 'Total'])

strongest_pokemon_per_type = get_strongest_per_type(pokemon_df)
print("Strongest Pokémon of Each Type:")
print(strongest_pokemon_per_type)

# 2. Beste Angreifer
best_attackers = pokemon_df.sort_values('Attack', ascending=False).head(10)
print("\nBest Attackers:")
print(best_attackers[['Name', 'Attack', 'Type 1', 'Type 2']])


         #                     Name           Type Total   HP Attack Defense  \
0     0001                Bulbasaur   Grass Poison   318   45     49      49   
1     0002                  Ivysaur   Grass Poison   405   60     62      63   
2     0003                 Venusaur   Grass Poison   525   80     82      83   
3     0003   Venusaur Mega Venusaur   Grass Poison   625   80    100     123   
4     0004               Charmander           Fire   309   39     52      43   
...    ...                      ...            ...   ...  ...    ...     ...   
1210  1023               Iron Crown  Steel Psychic   590   90     72     100   
1211  1024    Terapagos Normal Form         Normal   450   90     65      85   
1212  1024  Terapagos Terastal Form         Normal   600   95     95     110   
1213  1024   Terapagos Stellar Form         Normal   700  160    105     110   
1214  1025                Pecharunt   Poison Ghost   600   88     88     160   

     Sp. Atk Sp. Def Speed             

Strongest Pokémon of Each Type:
        Type         Strongest Pokémon  Total
0      Grass    Sceptile Mega Sceptile    630
1       Fire    Groudon Primal Groudon    770
2      Water      Kyogre Primal Kyogre    770
3        Bug                  Genesect    600
4     Normal                    Arceus    720
5       Dark  Tyranitar Mega Tyranitar    700
6     Poison       Eternatus Eternamax   1125
7   Electric                    Zekrom    680
8     Ground    Groudon Primal Groudon    770
9        Ice       Kyurem White Kyurem    700
10     Fairy      Zacian Crowned Sword    700
11     Steel  Metagross Mega Metagross    700
12  Fighting      Mewtwo Mega Mewtwo X    780
13   Psychic      Mewtwo Mega Mewtwo Y    780
14      Rock      Diancie Mega Diancie    700
15     Ghost     Giratina Origin Forme    680
16    Dragon       Eternatus Eternamax   1125
17    Flying    Rayquaza Mega Rayquaza    780

Best Attackers:
                         Name  Attack   Type 1    Type 2
201      Mewtwo Mega